<a href="https://colab.research.google.com/github/cdtlaura/nlp2/blob/main/Final_project_Document_Summarization_and_Translation_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

"Document Summarization and Translation App". This app summarizes lengthy documents and translates the summaries into multiple languages (e.g., French, Spanish, Chinese). Businesses could use it for:
Quickly understanding key points of reports.
Communicating summaries to international teams.

Modified Application Plan

Input: A user uploads a document (e.g., Word .docx) via a file uploader.

Processing:
Extract content from the uploaded document.
Summarize the document using an LLM (Together AI in this case).

Translate the summary into multiple languages (e.g., French, Spanish, Chinese).

Output:
Display:
The original summary.

Translations of the summary.

Optionally save the summary and translations as a downloadable file (e.g., PDF or .txt).

Explanation of the Workflow
Upload Document:


Uses Colab's files.upload() to allow users to upload a Word document.

Extract Content:


The uploaded document is read and its text content is extracted using the Docx2txtLoader from langchain_community.

Summarize Content:


The full document content is summarized using Together AI's meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo model.
The prompt explicitly requests a detailed and comprehensive summary.

Translate Summary:


Translations are handled by MarianMT models for French, Spanish, and Chinese.

Display Results:


The original summary and translations are displayed in Markdown format for readability.

Save Results:


The summary and translations are saved to a .txt file, which is then made available for download.

In [ ]:
!pip install together -qqq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 3.1 MB/s eta 0:00:00


In [ ]:
!pip install langchain_together python-dotenv langchain-community langchain youtube_transcript_api pytube numpy -qqq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.3/622.3 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.6 MB/s eta 0:00:00


In [ ]:
%pip install --upgrade --quiet  docx2txt -qqq

  Preparing metadata (setup.py) ... done


In [ ]:
!pip install langchain-together --upgrade -qqq


In [ ]:
# Import necessary libraries
from langchain_community.document_loaders import Docx2txtLoader
from transformers import MarianMTModel, MarianTokenizer
from IPython.display import Markdown, display
import os
from together import Together
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Set your API key as an environment variable
os.environ["TOGETHER_API_KEY"] = ""
client = Together()

In [ ]:
# Step 1: Load the Word document
loader = Docx2txtLoader("/content/Computer Vision10.docx")
data = loader.load()

# Step 2: Extract content from the document
document_content = " ".join([doc.page_content for doc in data])

# Step 3: Query the LLM for a summary
stream = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    messages=[{"role": "user", "content": f"Please summarize the following document:\n\n{document_content}"}],
    stream=True,
)

# Step 4: Collect the streamed summary (no printing here)
summary_text = ""  # Initialize an empty string to store the final summary
for chunk in stream:
    chunk_content = chunk.choices[0].delta.content or ""
    summary_text += chunk_content  # Concatenate the chunk content to build the summary

# Step 5: Display the summary only once
print("Summary:")
display(Markdown(summary_text))

# Step 6: Load MarianMT models and tokenizers for multiple translations
translation_models = {
    "French": {
        "model": MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-fr"),
        "tokenizer": MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr"),
    },
    "Spanish": {
        "model": MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-es"),
        "tokenizer": MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-es"),
    },
    "Chinese": {
        "model": MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-zh"),
        "tokenizer": MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-zh"),
    },
}

# Step 7: Function to split text into smaller chunks
def split_text(text, max_chunk_size=512):
    """
    Splits text into smaller chunks that fit within the token limit.
    """
    words = text.split()
    chunks = []
    current_chunk = []

    for word in words:
        current_chunk.append(word)
        if len(" ".join(current_chunk)) > max_chunk_size:
            chunks.append(" ".join(current_chunk))
            current_chunk = []

    if current_chunk:  # Add any remaining words as the last chunk
        chunks.append(" ".join(current_chunk))

    return chunks

# Step 8: Function to translate text in chunks
def translate_text_in_chunks(text, model, tokenizer, max_tokens=512):
    """
    Translates large text by splitting it into chunks that fit within the token limit.
    """
    chunks = split_text(text, max_chunk_size=max_tokens)  # Split text into chunks
    translated_chunks = []

    for chunk in chunks:
        input_ids = tokenizer.encode(chunk, return_tensors="pt", truncation=True)
        output_ids = model.generate(input_ids, max_length=max_tokens, num_beams=4)
        translated_chunks.append(tokenizer.decode(output_ids[0], skip_special_tokens=True))

    return " ".join(translated_chunks)

# Step 9: Translate the LLM summary into multiple languages
translations = {}
for language, resources in translation_models.items():
    translations[language] = translate_text_in_chunks(summary_text, resources["model"], resources["tokenizer"])



Summary:


The document is an introduction to image smoothing and thresholding techniques using OpenCV, a widely used open-source computer vision library. The authors, Laura Castillo and Yamil Guevara, provide an overview of the following techniques:

1. **Averaging Smoothing**: This technique reduces image noise by calculating the mean of pixel values within a kernel. It is simple and effective, but can lead to a blurred effect.
2. **Gaussian Blurring**: This technique improves on averaging smoothing by applying a Gaussian function to the pixels, emphasizing the central pixels in the kernel area. It is useful for reducing Gaussian noise in images.
3. **Median Blurring**: This technique is ideal for removing "salt-and-pepper" noise from images by calculating the median of all pixels within a kernel. It preserves edges and reduces the impact of outliers.
4. **Bilateral Filtering**: This technique smooths an image while preserving its edges by considering both the spatial distance and intensity difference of pixels in the kernel.
5. **Simple Thresholding**: This technique separates the foreground from the background by setting pixel values above or below a certain threshold to a maximum or minimum intensity.
6. **Adaptive Thresholding**: This technique extends simple thresholding by allowing the threshold value to vary across different regions of the image, making it effective for images with varying illumination.

The authors explain how to use OpenCV functions to implement these techniques, including:

* `cv.blur()` and `cv.boxFilter()` for averaging smoothing
* `cv.GaussianBlur()` for Gaussian blurring
* `cv.medianBlur()` for median blurring
* `cv.bilateralFilter()` for bilateral filtering
* `cv.threshold()` for simple thresholding
* `cv.adaptiveThreshold()` for adaptive thresholding

The document concludes that mastering these fundamental techniques is crucial for developing sophisticated computer vision applications, demonstrating the utility and flexibility of OpenCV in handling diverse vision tasks.

In [ ]:
# Step 10: Display the translated summaries
print("\nTranslated Summaries:")
for language, translated_text in translations.items():
    print(f"\nTranslated Summary in {language}:")
    display(Markdown(translated_text))


Translated Summaries:

Translated Summary in French:


Les auteurs, Laura Castillo et Yamil Guevara, donnent un aperçu des techniques suivantes : 1. **Averaging Smoothing**: Cette technique réduit le bruit d'image en calculant la moyenne des valeurs de pixel à l'intérieur d'un noyau. Elle est simple et efficace, ce qui en fait un point de départ commun pour la réduction du bruit d'image. 2. **Gaussian Blurring**: Cette technique améliore la moyenne par l'utilisation de l'image. Il est utile pour réduire le bruit gaussien dans les images et est largement utilisé dans les étapes de prétraitement pour la détection des bords et la segmentation de l'image. 3. **Blurring Median**: Cette technique est idéale pour éliminer le bruit "sel et poivre" des images en calculant la médiane de tous les pixels à l'intérieur d'un noyau. Il préserve les bords et réduit l'impact des valeurs aberrantes, ce qui le rend utile pour les applications où le maintien des détails structuraux est 4. ** Filtrage bilatéral**: Cette technique lisse une image tout en préservant ses bords en tenant compte à la fois de la distance spatiale et de la différence d'intensité des pixels dans le noyau. Elle est intensive par calcul mais convient à des applications comme la reconnaissance faciale et l'extraction des fonctionnalités. 5. **Simple Thresholding**: Cette technique sépare le premier plan de l'arrière-plan en fixant des valeurs de pixel au-dessus ou au-dessous d'un certain seuil à une intensité maximale ou minimale. 6. **Thresholding adaptatif**: Cette technique étend le concept de seuil simple en permettant à la valeur seuil de varier d'une région à l'autre de l'image. Elle est efficace pour les images avec un éclairage variable et fournit une meilleure précision de segmentation par rapport au seuil simple. Les auteurs concluent qu'OpenCV fournit un ensemble polyvalent d'outils pour le lissage et le seuil d'image, chacun avec La maîtrise de ces techniques fondamentales est cruciale pour développer des applications de vision informatique sophistiquées, démontrant l'utilité et la flexibilité d'OpenCV dans la gestion de diverses tâches de vision.


Translated Summary in Spanish:


El documento es una introducción a las técnicas de suavizado y umbralado de imágenes utilizando OpenCV, una biblioteca de visión de código abierto ampliamente utilizada. Los autores, Laura Castillo y Yamil Guevara, proporcionan una visión general de las siguientes técnicas: 1. **Average Smoothing**: Esta técnica reduce el ruido de la imagen calculando la media de los valores de píxeles dentro de un núcleo. Es simple y eficaz, lo que lo convierte en un punto de partida común para la reducción del ruido de imagen. 2. **Gaussian Blurring**: Esta técnica mejora en el promedio por aplicación de una función gaussiana a los píxeles, haciendo hincapié en los píxeles centrales en el área del núcleo. Es útil para reducir el ruido gaussiano en imágenes y se utiliza ampliamente en los pasos de preprocesamiento para la detección de bordes y la segmentación de imágenes. 3. **Blurring medio**: Esta técnica es ideal para eliminar el ruido "sal y pimienta" de las imágenes mediante el cálculo de la mediana de todos los píxeles dentro de un núcleo. 4. ** Filtrado bilateral**: Esta técnica suaviza una imagen al tiempo que preserva sus bordes considerando tanto la distancia espacial como la diferencia de intensidad de píxeles en el núcleo. Es computacionalmente intensiva pero adecuada para aplicaciones como reconocimiento facial y extracción de características. 5. ** Sencillo Thresholding**: Esta técnica separa el primer plano del fondo estableciendo valores de píxeles por encima o por debajo de un determinado umbral a una intensidad máxima o mínima. Es ideal para imágenes consistentes. La iluminación y es útil en escenarios donde el objetivo es crear una representación binaria de una imagen. 6. **Adaptive Thresholding**: Esta técnica amplía el concepto de umbral simple permitiendo que el valor de umbral varíe entre diferentes regiones de la imagen. Es eficaz para imágenes con iluminación variable y proporciona una mejor precisión de segmentación en comparación con el umbral simple. Los autores concluyen que OpenCV proporciona un conjunto versátil de herramientas para suavizar y umbralizar la imagen, cada una con Dominar estas técnicas fundamentales es crucial para desarrollar aplicaciones sofisticadas de visión por ordenador, demostrando la utilidad y flexibilidad de OpenCV en el manejo de diversas tareas de visión.


Translated Summary in Chinese:


本文介绍使用开放源开源计算机视觉图书馆OpenCV的图像平滑和临界技术。作者Laura Castillo和Yamil Guevara概述了以下技术:1. ** 虚拟平滑**:这种技术通过计算内核内像素值的平均值来减少图像噪音。它简单而有效,使它成为减少图像噪音的共同起点。 ** Gaussian Blurring**:这种技术在平均率方面有所改进。 a Gaussian 函数应用于像素, 突出内核区域的核心像素。 它有助于减少图像中的高萨噪音, 并广泛用于边缘探测和图像分割的预处理步骤。 ** ** Median Blurring **: 这一技术最理想的办法是通过计算内核内核中所有像素的中位数,从图像中去除“ 盐和比珀” 噪音。 它保存边缘并减少外部线的影响, 从而在保持结构细节时对应用程序有用。 4. ** 双边过滤**:这一技术既考虑到内核像素的空间距离和强度差异,又考虑到内核中像素的空间距离和强度差异,使图像在保持边缘的同时平滑,它具有计算强度,但适合面部识别和特征提取等应用。 6 ** 加速推进**:这一技术扩大了简单门槛值的概念,允许图像在不同区域之间的临界值变化,从而扩展了简单门槛值的概念。该技术对不同照明度的图像有效,与简单门槛值相比,提供了更好的分解准确度。作者得出结论,OpenCV为图像平滑和阈值提供了一套多功能工具,每个工具都有: 掌握这些基本技术对于开发先进的计算机视觉应用至关重要,可以证明开放文化电视公司在处理各种视觉任务时的效用和灵活性。

"""
### Business Use Case:
This application addresses the need for quick and efficient processing of lengthy documents in multinational organizations. By summarizing and translating business reports, research papers, and other documents, it enhances global communication and reduces manual workload.

### Business Value:
The tool enables businesses to save time, improve productivity, and foster collaboration across language barriers. By automating summarization and translation, organizations can focus on strategic decision-making while ensuring all team members have access to the same information, regardless of language.
"""
